In [ ]:
import numpy as np
import os
import plotly.graph_objects as go

from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results
from chimera_fgo.util.plot import plot_trajectories

%load_ext autoreload
%autoreload 2

In [ ]:
MAX_BIASES = [50, 100, 200]

In [ ]:
kitti_seq = '0034'
start_idx = 1550 if kitti_seq == '0028' else 0

# Load ground truth
gtpath = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

In [ ]:
TRAJLEN = 2000
gps_spoofing_biases = np.zeros(TRAJLEN)  
attack_start_idx = 1000
gps_spoofing_biases[attack_start_idx:] = np.linspace(0, 1, TRAJLEN-attack_start_idx)  # Ramping attack

spoof_pos_1 = gt_enu[:TRAJLEN].copy()
spoof_pos_1[:,0] += 50 * gps_spoofing_biases
spoof_pos_2 = gt_enu[:TRAJLEN].copy()
spoof_pos_2[:,0] += 100 * gps_spoofing_biases
spoof_pos_3 = gt_enu[:TRAJLEN].copy()
spoof_pos_3[:,0] += 200 * gps_spoofing_biases

gt_traj = go.Scatter(x=gt_enu[:TRAJLEN,0], y=gt_enu[:TRAJLEN,1], hovertext=np.arange(TRAJLEN), name='Ground-truth', line=dict(color='black'))
spoof_traj_1 = go.Scatter(x=spoof_pos_1[:,0], y=spoof_pos_1[:,1], hovertext=np.arange(TRAJLEN), name='0.5 m/s spoofed', line=dict(color='red', dash='dot'))
spoof_traj_2 = go.Scatter(x=spoof_pos_2[:,0], y=spoof_pos_2[:,1], hovertext=np.arange(TRAJLEN), name='1.0 m/s spoofed', line=dict(color='red', dash='dashdot'))
spoof_traj_3 = go.Scatter(x=spoof_pos_3[:,0], y=spoof_pos_3[:,1], hovertext=np.arange(TRAJLEN), name='2.0 m/s spoofed', line=dict(color='red', dash='dash'))

start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='black'), showlegend=False)
spoof_start = go.Scatter(x=[gt_enu[attack_start_idx,0]], y=[gt_enu[attack_start_idx,1]], 
            name='Spoofing start', mode='markers', marker=dict(size=10, color='red'), showlegend=True)
fig = go.Figure(data=[gt_traj, spoof_traj_1, spoof_traj_2, spoof_traj_3, start, spoof_start])
fig.update_layout(width=1000, height=1000, xaxis_title='East [m]', yaxis_title='North [m]')

fig.update_layout(font=dict(size=26))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )

if kitti_seq == '0018':
    fig.update_xaxes(range=[-300, 280])
    fig.update_layout(legend=dict(x=0.05, y=0.98))
elif kitti_seq == '0027':
    fig.update_xaxes(range=[-50, 450])
    fig.update_layout(legend=dict(x=0.68, y=0.98))
elif kitti_seq == '0028':
    fig.update_xaxes(range=[-20, 500])
    fig.update_layout(legend=dict(x=0.68, y=0.98))
elif kitti_seq == '0034':
    fig.update_xaxes(range=[-50, 950])
    fig.update_layout(legend=dict(x=0.05, y=0.98))
# fig.update_xaxes(range=[-300, 280]) # 0018
# fig.update_layout(legend=dict(x=0.05, y=0.98))
# fig.update_xaxes(range=[-50, 950]) # 0027
# fig.update_xaxes(range=[-50, 950]) # 0028
# fig.update_xaxes(range=[-50, 950]) # 0034
fig.show()